In [1]:
import openmc
import openmc.deplete as od
import math
import pandas as pd
import numpy as np
import openmc.mgxs as mgxs

In [2]:
fuel = openmc.Material(name="uo2")
fuel.add_element("U", 1, percent_type="ao", enrichment=4.25)
fuel.add_element("O", 2)
#fuel.add_nuclide('I135',)
#fuel.add_nuclide('Xe135', )
#fuel.add_nuclide('Xe136', )
#fuel.add_nuclide('Cs135', )
#fuel.add_nuclide('Gd157', )
#fuel.add_nuclide('Gd156')
#fuel.set_density("g/cc", 10.4)

#fuel.set_density('sum')
#for nuc in ['I135','Xe135','Cs135','Gd157','Gd156']:
#    if nuc not in available_cross_sections:
#        continue
#    atoms = result[0, mat_id, nuc]
#    if atoms > 0.0:
#        atoms_per_barn_cm = 1e-24 * atoms / mat.volume
#        mat.remove_nuclide(nuc) # Replace if it's there
#        mat.add_nuclide(nuc, atoms_per_barn_cm)

clad = openmc.Material(name="clad")
clad.add_element("Zr", 1)
clad.set_density("g/cc", 6)

water = openmc.Material(name="water")
water.add_element("O", 1)
water.add_element("H", 2)
water.set_density("g/cc", 1.0)
water.add_s_alpha_beta("c_H_in_H2O")

radii = [0.42, 0.45]
fuel.volume = math.pi * radii[0] ** 2

materials = openmc.Materials([fuel, clad, water])

In [150]:
fuel.get_nuclide_atom_densities()

OrderedDict([('U234', ('U234', 8.922411359424316e-06)),
             ('U235', ('U235', 0.000998240191860822)),
             ('U238', ('U238', 0.022192386373095892)),
             ('U236', ('U236', 4.572419549506112e-06)),
             ('O16', ('O16', 0.04639065406771322)),
             ('O17', ('O17', 1.7588724018066158e-05))])

In [3]:
pin_surfaces = [openmc.ZCylinder(r=r) for r in radii]
pin_univ = openmc.model.pin(pin_surfaces, materials)
bound_box = openmc.rectangular_prism(1.24, 1.24, boundary_type="reflective")
root_cell = openmc.Cell(fill=pin_univ, region=bound_box)
geometry = openmc.Geometry([root_cell])

In [4]:
settings = openmc.Settings()
settings.particles = 1000
settings.inactive = 10
settings.batches = 50

In [5]:
groups = mgxs.EnergyGroups()
groups.group_edges = np.array([0.,20.0e6])
fission = mgxs.FissionXS(domain=fuel, groups=groups, by_nuclide=True)
capture = mgxs.CaptureXS(domain=fuel, groups=groups,by_nuclide=True)

tallies = openmc.Tallies()
tallies += fission.tallies.values()
tallies += capture.tallies.values()


In [6]:
model = openmc.model.Model(geometry, materials, settings, tallies)

In [7]:
model.run()

/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:67: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)


                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

PosixPath('/home/ooblack/projects/openmc-depletion-testing/statepoint.50.h5')

## Get 1G XS

In [8]:
# Load the last statepoint file
sp = openmc.StatePoint('statepoint.50.h5')


# Load the tallies from the statepoint into each MGXS object
fission.load_from_statepoint(sp)
capture.load_from_statepoint(sp)
# Close the statepoint file now that we're done getting info
sp.close()

In [132]:
fission_df = fission.get_pandas_dataframe(xs_type='micro')
capture_df = capture.get_pandas_dataframe(xs_type='micro')

In [133]:
capture_df.index=capture_df['nuclide']
fission_df.index=fission_df['nuclide']

In [135]:
capture_df.drop(['nuclide','material','group in','std. dev.'], axis=1, inplace=True)
fission_df.drop(['nuclide','material','group in','std. dev.'], axis=1, inplace=True)

In [137]:
capture_df.rename({'mean':'(n,gamma)'}, axis=1, inplace=True)
fission_df.rename({'mean':'fission'}, axis=1, inplace=True)

In [139]:
micro_xs = pd.concat([capture_df,fission_df], axis=1)

In [141]:
micro_xs.to_csv('micro_xs_simple.csv')

In [20]:
capture.tallies.values()

odict_values([Tally
	ID             =	8
	Name           =	
	Filters        =	MaterialFilter, EnergyFilter
	Nuclides       =	total
	Scores         =	['flux']
	Estimator      =	tracklength, Tally
	ID             =	9
	Name           =	
	Filters        =	MaterialFilter, EnergyFilter
	Nuclides       =	U234 U235 U238 U236 O16 O17
	Scores         =	['absorption']
	Estimator      =	tracklength, Tally
	ID             =	10
	Name           =	
	Filters        =	MaterialFilter, EnergyFilter
	Nuclides       =	U234 U235 U238 U236 O16 O17
	Scores         =	['fission']
	Estimator      =	tracklength])

In [ ]:
micro_xs.to

In [140]:
micro_xs.to_csv('')

'nuclide,"(n,gamma)",fission\nU234,23.518634203050645,0.49531930067650976\nU235,10.621118186344665,49.10955932965905\nU238,0.8652742788116043,0.10579281644765708\nU236,9.095623870006154,0.32315392339237936\nO16,0.0029535689693132015,0.0\nO17,0.05980775766572907,0.0\n'

In [17]:
fission.print_xs(xs_type='micro')

Multi-Group XS
	Reaction Type  =	fission
	Domain Type    =	material
	Domain ID      =	1
	Nuclide        =	U234
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	4.95e-01 +/- 6.42e-01%

	Nuclide        =	U235
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	4.91e+01 +/- 6.08e-01%

	Nuclide        =	U238
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	1.06e-01 +/- 8.73e-01%

	Nuclide        =	U236
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	3.23e-01 +/- 1.59e+00%

	Nuclide        =	O16
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	0.00e+00 +/- 0.00e+00%

	Nuclide        =	O17
	Cross Sections [barns]:
            Group 1 [0.0        - 20000000.0eV]:	0.00e+00 +/- 0.00e+00%





/home/ooblack/projects/openmc/openmc/tallies.py:1255: RuntimeWarning: invalid value encountered in true_divide
  data = self.std_dev[indices] / self.mean[indices]


In [136]:
capture_df

,mean
nuclide,
U234,23.518634
U235,10.621118
U238,0.865274
U236,9.095624
O16,0.002954
O17,0.059808
